V3.1 -  Moving Aircraft And Static Radar

In [1]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
#cbg
# Initialize libraries and functions PC
#%pip install --upgrade numpy
#import matplotlib.pyplot as plt
#from scipy.signal import find_peaks
#import cv2
#from prettytable import PrettyTable
import numpy as np
import math
import random
import time as timess
from IPython.display import clear_output, Markdown, display
from abc import ABC, abstractmethod

def db_to_normal(x):
    return 10**(x/10)

def normal_to_db(x):
    return 10*math.log10(x)

def pdw_tabular_print(pdw,time):
    if(len(pdw)>1):
        display(Markdown('{}<strong>{}</strong>{}'.format('PDW\'s of the recieved pulses in ',str(time//10)+' nano seconds',' are as follows')))
        t = PrettyTable(list(pdw[0].keys()))
        for i in pdw[:]:
            t.add_row(list(i.values()))
        print(t)


current_tick_ns = 0
delta_t_ns = 0
time_1ns=0
time_1ms_ns = 1000000
time_1us_ns = 1000


# base_noise_min=db_to_normal(-1000)
# base_noise_max=db_to_normal(-75)
base_noise_min=db_to_normal(-1000*1000)
base_noise_max=db_to_normal(-75*1000)
ci=0#len(time)//4
cf=10000
cf=10000
fig_x , fig_y = 25 , 2
clrs= 'rgbcmyk'
rand_skew=0
mult = 2

tick_in_ns      =0
previous_tick_ns=0
delta_tick_ns   =0




In [3]:
# # Initiallize tables
# rf = open("../Radar_G_table_3d.txt", "w")
# # f = open("../RWR_G_table_omni.txt", "w")
# for j in range(5,185,5):#frequency
#     for k in range(360):
#         # print(j,k)
#         # f.write(str(j/10)+' '+str(k))
#         # f.write(' | ')
#         rf.write(str(j/10)+' '+str(k))
#         rf.write(' | ')
#         for i in range(360):
#             if(((i>=0 and i<=90//mult) or (i>=360-(90//mult) and i<360)) and ((k>=0 and k<=90//mult) or (k>=360-(90//mult) and k<360))):
#                 # f.write(str(math.cos(math.radians(0))*math.cos(math.radians(0)))+' ')
#                 rf.write(str(math.cos(math.radians(mult*i))*math.cos(math.radians(mult*k)))+' ')
#             else:
#                 # f.write(str(0)+' ')
#                 rf.write(str(0)+' ')
#         # f.write('\n')
#         rf.write('\n')
# # f.close()
# rf.close()

In [4]:
# f = open("../angle_table.txt", "w")
# for th in range(-45,45):
#     a1=round(normal_to_db(math.cos(math.radians(45+th))),4)
#     a2=round(normal_to_db(math.cos(math.radians(45-th))),4)
#     f.write(str(round(a2-a1,5))+',')
# th=45
# a1=round(normal_to_db(math.cos(math.radians(45+th))),4)
# a2=round(normal_to_db(math.cos(math.radians(45-th))),4)
# f.write(str(round(a2-a1,5)))
# f.close()

In [5]:
# Initiallize tables

# rf = open("Radar_G_table.txt", "w")
# f = open("RWR_G_table.txt", "w")
# for j in range(5,185,5):
#     f.write(str(j/10))
#     f.write(' | ')
#     rf.write(str(j/10))
#     rf.write(' | ')
#     for i in range(360):
#         if((i>=0 and i<=90) or (i>=270 and i<360)):
#             f.write(str(math.cos(math.radians(i)))+' ')
#             rf.write(str(math.cos(math.radians(i)))+' ')
#         else:
#             f.write(str(0)+' ')
#             rf.write(str(0)+' ')
#     f.write('\n')
#     rf.write('\n')
# f.close()    
# rf.close()  

# f = open("angle_table.txt", "w")
# for th in range(-45,45):
#     a1=round(normal_to_db(math.cos(math.radians(45+th))),4)
#     a2=round(normal_to_db(math.cos(math.radians(45-th))),4)
#     f.write(str(round(a2-a1,5))+',')
# th=45
# a1=round(normal_to_db(math.cos(math.radians(45+th))),4)
# a2=round(normal_to_db(math.cos(math.radians(45-th))),4)
# f.write(str(round(a2-a1,5)))
# f.close()


RWR_G_table={}
f = open("RWR_G_table.txt", "r")
Lines = f.readlines()
for line in Lines:
    tmp=list(line.split())
    tmp_freq=float(tmp[0])
    tmp=tmp[2:]
    RWR_G_table[tmp_freq]=[]
    for radar in tmp:
        RWR_G_table[tmp_freq].append(float(radar))
f.close()

Radar_G_table={}
f = open("Radar_G_table.txt", "r")
Lines = f.readlines()
for line in Lines:
    tmp=list(line.split())
    tmp_freq=float(tmp[0])
    tmp=tmp[2:]
    Radar_G_table[tmp_freq]=[]
    for radar in tmp:
        Radar_G_table[tmp_freq].append(float(radar))
f.close()

f = open("angle_table.txt", "r")
angle_table=tuple(map(float,f.read().split(',')))
f.close()

In [6]:
class battle_system(ABC):
    def __init__(self):
        pass

    @abstractmethod
    def get(self):
        pass

    @abstractmethod
    def set(self):
        pass

    @abstractmethod
    def ontick(self):
        pass

    def __del__(self):
        pass

In [7]:
class platform(battle_system):
    def __init__(self, pos=(0,0,0),speed=0,onboard_systems=[]):
        super().__init__()
        if(len(pos)==2):
            self.position = list(pos)+[0]
        else:
            self.position = pos
        self.speed = speed
        self.onboard_systems = onboard_systems

    def __del__(self):
        super().__del__()

In [8]:
class aircraft(platform):
    def __init__(self, id=0,heading=0,speed=10,aircraftWaypoints=[],onboard_systems=[]):
        super().__init__(aircraftWaypoints[0] ,speed,onboard_systems)
        self.id = id                                       # Id of the Aircraft
        self.heading = heading                             # heading of the Aircraft in degrees
        for i in range(len(aircraftWaypoints)):
            if len(aircraftWaypoints[i])==2:
                aircraftWaypoints[i]=list(aircraftWaypoints[i])+[0]
        self.aircraftWaypoints=aircraftWaypoints
        self.next_waypoint=1

    def move_aircraft(self):
        if(len(self.aircraftWaypoints)>1):
            time_diff=1
            cur_pos=list(self.position)
            next_waypoint=self.next_waypoint
            d=math.dist(self.aircraftWaypoints[next_waypoint],cur_pos)
            idx=0
            while(d < (self.speed*time_diff)):
                idx += 1
                next_waypoint += 1
                if (next_waypoint== len(self.aircraftWaypoints)):
                    next_waypoint = 0
                if(idx>len(self.aircraftWaypoints)*2):
                    raise Exception("Aircraft speed is too much")
                d=math.dist(self.aircraftWaypoints[next_waypoint],cur_pos)
            heading=math.degrees(math.atan2(self.aircraftWaypoints[next_waypoint][1] - cur_pos[1],self.aircraftWaypoints[next_waypoint][0] - cur_pos[0]))
            while(heading>=360):
                heading-=360
            while(heading<0):
                heading+=360
            self.heading=heading
            cur_pos[0] += ((self.speed * time_diff) * (self.aircraftWaypoints[next_waypoint][0] - cur_pos[0]) / d)
            cur_pos[1] += ((self.speed * time_diff) * (self.aircraftWaypoints[next_waypoint][1] - cur_pos[1]) / d)
            self.position=cur_pos
            self.next_waypoint=next_waypoint

    def get(self):
        return self.position

    def set(self,speed=1):
        self.speed=speed

    def ontick(self):
        self.move_aircraft()

    def __del__(self):
        super().__del__()

In [9]:
class sensor(battle_system):
    def __init__(self,host_platform):
        super().__init__()
        self.host_platform = host_platform

    def __del__(self):
        super().__del__()

In [10]:
#clrdr
class radar(sensor):
    def __init__(self, id, pri=500, pwd=10, main_beam_azimuth=0, rpm=60,
                 host_platform=[]):
        super().__init__(host_platform)

        self.id = id                                        # Id of the radar
        self.beam_width = 10                                # Beam width in degrees
        self.Pt = (10**10)*1                                # Peak transmitted power
        self.position = [0, 0]                              # radar position (x,y)
        self.main_beam_azimuth = main_beam_azimuth          # azimuth of the radar in degrees
        self.rpm = rpm

        self.angle_from_radar_to_ac = 0
        self.ac_angle_start = 0
        self.ac_angle_end = 0
        self.duration_per_rotation_ns = 1.0 / (rpm / 60.0)     # rpm/60 is rps
        self.time_at_angle_start_ns = 0
        self.time_at_angle_end = 0

        # # If a host platform (e.g., an aircraft) with position is provided, compute initial angles
        # try:
        #     if self.host_platform:
        #         ac_pos = getattr(self.host_platform, "position", None)
        #         if ac_pos:
        #             self.angle_from_radar_to_ac = self.find_angle(ac_pos)
        #             self.ac_angle_start = self.angle_from_radar_to_ac - self.beam_width / 2
        #             self.ac_angle_end = self.angle_from_radar_to_ac + self.beam_width / 2
        #             self.time_at_angle_start = self.duration_per_rotation * (self.ac_angle_start % 360)
        #             self.time_at_angle_end = self.duration_per_rotation * (self.ac_angle_end % 360)
        # except Exception:
        #     pass

    def find_angle(self, ac_pos):
        # compute angle from radar position to aircraft position in degrees [0,360)
        dx = ac_pos[0] - self.position[0]
        dy = ac_pos[1] - self.position[1]
        angle = math.degrees(math.atan2(dy, dx)) % 360
        return angle

    def get(self):
        return self.position

    def set(self, speed=None, azimuth=None, elevation=0):
        if speed is not None:
            self.speed = speed
        if azimuth is not None:
            self.azimuth = azimuth
        self.elevation = elevation

    # keep ontick signature compatible with callers; implement behavior if needed
    def ontick(self):
        # placeholder: advance main_beam_azimuth based on rpm and an external timestep if desired
        delta_tick_ns = tick_in_ns - previous_tick_ns
        self.main_beam_azimuth = (self.main_beam_azimuth + (360.0 * self.rpm / 60.0) * (delta_tick_ns / 1000000000.0)) % 360
        previous_tick_ns = tick_in_ns
        pass

    def __del__(self):
        super().__del__()

In [11]:
class rwr(sensor):
    def __init__(self, number_of_antennas,host_platform):
        super().__init__(host_platform)
        self.number_of_antennas=number_of_antennas
        self.threats=[]
        self.pdws=[]
        self.emitter_records=[]
        self.recieved_power=[]
        self.tempAngleDiff = 0

    def get(self):
        return self.recieved_power

    def set(self,PowerAtRWR):
        self.recieved_power = PowerAtRWR

    def ontick(self):
        pass

    def __del__(self):
        super().__del__()

In [12]:
class dtse:
    def __init__(self, aircraft, radar, rwr,
                 aircraft_speed=0.5, radar_rot_speed=5, radar_speed=0.1,
                 radar_mode="circular", conical_theta_speed=2, conical_rad=5, conical_dis=10,
                 raster_elevation=60, raster_speed=5, palmer_raster_speed=20,
                 palmer_azimuth=-30, palmer_azimuth_speed=10):
        self.aircraft = aircraft
        self.radar = radar  # Changed to a list of radars
        self.rwr = rwr

        # Simulation parameters for dtse to manage
        self.aircraft_speed = aircraft_speed
        self.radar_rot_speed = radar_rot_speed
        self.radar_speed = radar_speed
        self.radar_mode = radar_mode

        # State variables for radar modes (initialized to match original code's behavior)
        self.clockWise = True
        self.raster_clockWise = True
        self.conical_theta = 0
        self.conical_rad = conical_rad
        self.conical_dis = conical_dis
        self.conical_theta_speed = conical_theta_speed
        self.raster_elevation = raster_elevation
        self.raster_speed = raster_speed
        self.palmer_raster_speed = palmer_raster_speed
        self.palmer_azimuth = palmer_azimuth
        self.palmer_azimuth_speed = palmer_azimuth_speed
        self.spiral_increse = True  # For spiral mode
        self.conical_rad_spiral = conical_rad  # For spiral mode

        # Additional RWR related parameters that were global
        self.RiseState = []
        self.FallState = []
        self.pri = 123
        self.pulseWidth = self.pri - 0
        self.StateFlag1 = 0
        self.StateFlag2 = 0
        self.StateFlag3 = 0
        self.tolerance = 0
        self.shiftCurrent = 0
        self.shiftPrevious = 0
        self.missingCount = 0
        self.ResetFlag = 1 # Assuming ResetFlag is also an instance variable

    def fuse(self):
        global radar_Pt,radar_frequency,radar_position,radar_heading,aircraft_position,aircraft_heading, receivers_n, PowerAtRWR
        # This method is intended for processing and integrating sensor data.
        # For now, it's a placeholder. Future enhancements could include:
        # - Analyzing RWR's recorded_power for emitter identification.
        # - Estimating radar parameters (PRI, PW, AOA) based on RWR data.
        # - Updating an internal threat awareness picture.
        
        # get aircraft location, radar location

        R = math.dist(radar_position,aircraft_position)      # Distance between the radar and the aircraft

        # Lambda=3*(10**8)/radar_frequency
        Lambda=2


        angle = math.degrees(math.atan2(self.radar.position[1]-self.aircraft.position[1],
                                      self.radar.position[0]-self.aircraft_position[0]))
        - self.aircraft.heading
        
        # angle_rad = radar_heading
        # angle_t=int(angle-angle_rad-180-(360-aircraft.heading))
        # if(angle_t>=360):
        #     angle_t-=360                                     # Angle between the radar's boresight and aircraft
        # if(angle_t<0):
        #     angle_t+=360

        
        P=(radar_Pt*Radar_G_table[radar_frequency][angle_t]*(Lambda**2))/((4*math.pi*R)**2)  # Power recieved by the aircraft

        rwr_angles=[]

        for j in range(receivers_n):                          # front left, rear left, rear right and front right respectively
            rwr_angles.append((360/receivers_n)*(1+2*j)/2) 

        PowerAtRWR = []
        for idx in range(receivers_n):
            PowerAtRWR.append(abs(P*RWR_G_table[radar_frequency][int(angle-rwr_angles[idx])]))
        
        print("Power at RWR: ", PowerAtRWR)

        # calculate rx gain and received power
        # calculate incidence angle on aircraft and hance angle with each rwr antenna
        # calculate rx power in each rwr antenna. 
        # generate PDW with A1, A2, A3, A4
        
        #THIS IS RWR LOGIC
        # based on max 2 antenna determine azimuth of radar
        # do this for beam sweeping through aircraft and mark a point when it is sweeping (typically A1 = A2)
        # observe the time difference for successive visits to the marked point. That gives scan rate or RPM
        


    def driver_run(self, iterations=5600):
        for idx in range(iterations):

            # --------- Get -------------
            self.aircraft.get()
            self.rwr.get() # Get the recorded power
            self.radar.get()
            # --------- Get -------------


            radar_position = self.radar.position
            radar_heading = self.radar.main_beam_azimuth
            aircraft_position = self.aircraft.position
            aircraft_heading = self.aircraft.heading


            #generate PDWs
            self.fuse()
            print("Time: ", idx," | Power at RWR: ", self.rwr.recieved_power)


            if (tick_in_ns  >= radar.time_at_angle_start_ns and tick_in_ns < radar.time_at_angle_end) :
                #aircraft is within beam
                tick_in_ns = tick_in_ns + 1                
            else:              
		        #aircraft is outside beam
                radar.time_at_angle_start_ns += radar.duration_per_rotation_ns
                radar.time_at_angle_end += radar.duration_per_rotation_ns
                tick_in_ns=radar.time_at_angle_start_ns
                mbzim = radar.ac_angle_start
            

            # --------- Set -------------
            self.aircraft.set(self.aircraft_speed)
            self.rwr.set(PowerAtRWR)#send the generated PDWs to RWR
            self.radar.set()
            # --------- Set -------------


            # --------- Ontick -------------
            self.aircraft.ontick()
            self.rwr.ontick()
            # Radar movement is now handled solely by the radar's own ontick method,
            # without mode-specific adjustments within the dtse driver_run.
            self.radar.ontick()
            # --------- Ontick -------------


In [13]:
# --- Scenario Setup for dtse testing ---

# 1. Initialize Aircraft
# Setting up a stationary aircraft for simplicity in this test.
#cbm
aircraft_test_waypoints = [(700, 700, 0)]
aircraft = aircraft(id=0,
                      heading=0,
                      speed=0, # Stationary for this test
                      aircraftWaypoints=aircraft_test_waypoints)

# 2. Initialize RWR (onboard the aircraft)
rwr = rwr(number_of_antennas=4, host_platform=aircraft)
aircraft.onboard_systems = [rwr]

# 3. Initialize Radar
# Placing the radar close to the aircraft for visibility.
radar_test_waypoints = [[(750, 700, 0)]]
radar_test = radar(id=1,
                   pri=2000,
                   pwd=1000,
                    #Initial azimuth                   
                   )
radar_test.freq = 0.5


# 4. Initialize dtse instance
# Setting radar_mode to 'circular' as requested.
dtse_instance = dtse(aircraft,
                       radar_test,
                       rwr,
                       aircraft_speed=0, # Aircraft is stationary
                       radar_mode="circular", # Circular scan mode
                       radar_rot_speed=30 # Adjust rotation speed for visible change in 10 iterations
                      )


antenna_length = 10
rwr_antenna_n = 4              # Number of recievers on the aircraft(4 or 6)
radars_n = 3                   # Number of radars
receivers_n = 4 # Effective number of antenna


radar_Pt = radar_test.Pt # Peak transmitted power
radar_frequency = radar_test.freq #GHz
radar_position = radar_test.position
#radar_heading = radar_test.azimuth
aircraft_position = aircraft.position
aircraft_heading = aircraft.heading
PowerAtRWR = [0, 0, 0, 0] # Initialize PowerAtRWR for the test


# 5. Run dtse.driver_run for 10 iterations
print("Starting dtse simulation for 10 iterations...")
dtse_instance.driver_run(iterations=10)
print("dtse simulation finished.")

Starting dtse simulation for 10 iterations...


ValueError: both points must have the same number of dimensions

In [ ]:
# Initialization
rwr_antenna_n = 4              # Number of recievers on the aircraft(4 or 6)
radars_n = 3                   # Number of radars
receivers_n = 4 # Effective number of antenna

img = np.zeros((1400,1400,3), np.uint8)
# Aircraft object is created
#straight line
aircraft_waypoints = [(1100, 1400), (1100, 00)]
#Circle
# aircraft_waypoints = [(100, 1400), (100, 1377), (100, 1354), (101, 1331), (103, 1309), (104, 1286), (107, 1264), (109, 1241), (112, 1219), (116, 1196), (119, 1174), (123, 1151), (128, 1129), (133, 1107), (138, 1085), (144, 1063), (150, 1041), (156, 1019), (163, 998), (170, 976), (178, 955), (186, 934), (194, 913), (203, 892), (212, 871), (221, 850), (231, 830), (241, 809), (252, 789), (262, 769), (274, 750), (285, 730), (297, 711), (309, 691), (322, 673), (335, 654), (348, 635), (361, 617), (375, 599), (389, 581), (404, 564), (418, 547), (433, 530), (449, 513), (464, 496), (480, 480), (496, 464), (513, 449), (530, 433), (547, 418), (564, 404), (581, 389), (599, 375), (617, 361), (635, 348), (654, 335), (673, 322), (691, 309), (711, 297), (730, 285), (749, 274), (769, 262), (789, 252), (809, 241), (830, 231), (850, 221), (871, 212), (892, 203), (913, 194), (934, 186), (955, 178), (976, 170), (998, 163), (1019, 156), (1041, 150), (1063, 144), (1085, 138), (1107, 133), (1129, 128), (1151, 123), (1174, 119), (1196, 116), (1219, 112), (1241, 109), (1264, 107), (1286, 104), (1309, 103), (1331, 101), (1354, 100), (1377, 100)]

#complex
# aircraft_waypoints = [(336, 630), (171, 629), (167, 630), (157, 629), (151, 628), (148, 626), (144, 623), (42, 525), (38, 522), (32, 518), (29, 514), (27, 509), (27, 504), (28, 500), (30, 495), (33, 493), (45, 485), (70, 468), (265, 288), (273, 282), (275, 274), (275, 269), (273, 156), (276, 150), (281, 146), (285, 145), (289, 145), (296, 144), (615, 145), (619, 146), (626, 149), (630, 155), (633, 160), (632, 165), (633, 171), (634, 179), (633, 184), (633, 189), (633, 195), (636, 295), (636, 298), (636, 305), (636, 312), (638, 320), (640, 327), (643, 349), (648, 402), (649, 415), (649, 424), (652, 436), (652, 447), (654, 455), (656, 461), (656, 466), (655, 471), (656, 479), (655, 487), (656, 501), (654, 557), (656, 565), (652, 575), (648, 585), (638, 593), (623, 605), (616, 609), (608, 612), (595, 614), (589, 615), (578, 615), (563, 613), (553, 610), (545, 608), (533, 601), (522, 593), (515, 585), (505, 570), (500, 559), (497, 545), (497, 534), (499, 524), (503, 515), (511, 505), (520, 497), (534, 492), (546, 484), (556, 478), (574, 474), (584, 472), (597, 470), (611, 468), (627, 467), (639, 465), (655, 466), (659, 466), (683, 466), (692, 466), (723, 462), (729, 460), (740, 454), (751, 444), (757, 438), (1099, 109), (1105, 104), (1114, 99), (1121, 96), (1132, 93), (1147, 93), (1166, 93), (1177, 96), (1183, 100), (1188, 107), (1189, 112), (1191, 124), (1194, 140), (1224, 505), (1225, 514), (1225, 522), (1225, 526), (1225, 532), (1225, 547), (1226, 559), (1225, 572), (1231, 615), (1229, 626), (1226, 637), (1217, 649), (1205, 655), (1186, 660), (1165, 662), (1152, 664)]

aircraft_0 = aircraft(id=0,
                      heading=random.uniform(0,360),
                      speed=10,
                      aircraftWaypoints=aircraft_waypoints[:])

# RWR object is created

rwr_0 = rwr(number_of_antennas=receivers_n,host_platform=aircraft_0)
aircraft_0.onboard_systems=[rwr_0]
# (170, 1329)
# (170, 1470)
# (29, 1470)
# (29, 1329)
# RADAR object is created
radar_positions = np.array([
                            (1350.0, 700.0),
                            (39.0, 500.0),
                            (50.0, 42.0),
                            (1355.0, 670.0),
                           ])
radar_waypoints = [[]]*len(radar_positions)
radar_waypoints = list([
                        [radar_positions[0]]#+[(829, 384), (505, 383)]
                    #    ,[radar_positions[1]]
                    #    ,[radar_positions[2]]
                    #    ,[radar_positions[3]]
                       ])
# radar_waypoints = list([[radar_positions[0]]])
radars_n=len(radar_waypoints)
radar=[]
#'fixed'
#'skew'
#'jittered'
#'staggered'
#'switcher'
signatures=[['fixed'],
            ['fixed'],
            ['fixed'],
            ['fixed']]
# signature=['switcher',[[2,80],[4,150],[3,400]]]
# signature=['staggered',[50,100,200,400]]
# signature=['jittered']
# signature=['skew']
# signature=['fixed']
# signatures=[]
# for i in range(radars_n):
#     signatures.append(signature)
pris=[2000,4500,5500,7200]
for j in range(radars_n):
    heading_angle=int(math.degrees(math.atan2(aircraft_0.position[1]-radar_positions[j][1],aircraft_0.position[0]-radar_positions[j][0])))+30
    heading_noise_range=0
    rand_pri=int(random.uniform(100,1000))
    radar_j = radar(id=j+1,pri=pris[j],pwd=1000,
                    azimuth=random.randint(heading_angle-heading_noise_range,heading_angle+heading_noise_range),
                    radarWaypoints=radar_waypoints[j],
                    signature_type=signatures[j]
                   )
    radar_j.elevation=325
    rand_freq=(int(random.uniform(0,18)))
    # radar_j.freq=0.5+rand_freq
    radar_j.freq=0.5
    radar.append(radar_j)
    del radar_j

# Visualization
print('Aircraft and it\'s description')
t = PrettyTable(['Id','Number of RWR antenna\'s'])
i=aircraft_0
t.add_row([i.id+1,i.onboard_systems[0].number_of_antennas])
print(t)
print('Radars and their description')
t = PrettyTable(['Id', 'Peak transmitted power(db)','frequency (GHZ)','Pulse Repetition Interval(100 pico seconds)','Pulse Width(100 pico seconds)','signature_type'])
for i in radar:
    t.add_row([i.id,10*math.log10(i.Pt),i.freq,i.pri,i.pwd,i.signature_type])
print(t)

In [ ]:
# Main Function

slope=1

#Initialization
radar_mode="circular"
# radar_mode="sector"
# radar_mode="helical"
# radar_mode="raster"
# radar_mode="conical"
# radar_mode="spiral"
# radar_mode="palmer"
# radar_mode="palmerRaster"
# radar_mode="tracking"
search_radars={1}
play_back_speed=1
play_flag=0
resize=1.5
cv2.namedWindow("Visualisation", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Visualisation", int(img.shape[1]/resize), int(img.shape[0]/resize))
tmp_s_r=20
tmp_s_a=20

time=(10**4)*1 #in 1 nano seconds
# time=55 #in 1 nano seconds

visualize=True
idx=0
pdws=[]
pulse_train=[]
pulse_train_f=[]
for j in range(receivers_n):
    pulse_train.append([])
    for i in range(len(radar)):
        pulse_train[-1].append([])
    pulse_train_f.append([])
Emitter_Track_File={}
full_Emitter_Track_File={}
backup_ageinout_Emitter_Track_File={}
backup_actual_Emitter_Track_File={}
rise_fall_flag={}
serach_angles=[]
sn=3
for i in range(sn):
    serach_angles.append(360*i//sn)
serach_angles.insert(1,10)
serach_angles.insert(len(serach_angles),350)
age_in_PFM ={1:3,
             2:3,
             3:5,
             4:5}
age_out_PFM={1:3,
             2:3,
             3:5,
             4:3}

antenna_length = 10
A1_t_table = []
A2_t_table = []
A3_t_table = []
A4_t_table = []
Ax_t_table = []
Ay_t_table = []
Rx_t_table = []
Ry_t_table = []
Ra_t_table = []

# aircraft_0.heading = -50

aircraft_speed = .5
radar_rot_speed= 5
radar_speed = .1
conical_theta_speed=2
conical_theta=0
conical_rad=5
conical_dis=10
clockWise=True
raster_clockWise=True
raster_elevation=60
raster_speed=5
palmer_raster_speed=20
conical_rad_spiral=conical_rad
palmer_azimuth=-30
spiral_increse=True
palmer_azimuth_speed=10

# Adaptive Search Regime Variables
beta = 0   # Radar azimuth at t=0
alpha = 360 # Beam Width
omega = radar_rot_speed # Radar Angular velocity
thetaOffset = 0

RiseState = []
FallState = []
pri = 123
pulseWidth = pri-0
StateFlag1 = 0
StateFlag2 = 0
StateFlag3 = 0
tolerance = 0
shiftCurrent = 0
shiftPrevious = 0
Analysis1 = []
Analysis2 = []
Analysis3 = []
Analysis4 = []
Analysis5 = []
# Main Loop
while(idx<5600):
    print('tick number - ',str(idx+1))
    # Get,Set,Ontick of aircraft
    # print(aircraft_0.get())
    t_pos=aircraft_0.get()
    airc_pos=(int(t_pos[0]),int(t_pos[1]))
    Ax_t_table.append(airc_pos[0])
    Ay_t_table.append(airc_pos[1])
    aircraft_0.set(aircraft_speed)
    aircraft_0.ontick()

    tmp_i=0
    for i in radar:
        t_pos=i.get()
        radar_pos=(int(t_pos[0]),int(t_pos[1]))
        heading_angle=int(math.degrees(math.atan2(airc_pos[1]-radar_pos[1],airc_pos[0]-radar_pos[0])))#+180
        heading_noise_range=0
        if (radar_mode=="circular"):
            clockWise=False
            if(idx < 510 or idx > 600 or True):
                i.set(radar_speed,((i.azimuth+radar_rot_speed)%360) if clockWise else((i.azimuth-radar_rot_speed)%360) ,0)
        elif (radar_mode=="sector"):
            min_sector=30
            max_sector=250
            if(i.azimuth<min_sector+radar_rot_speed and i.azimuth>min_sector-radar_rot_speed):
                clockWise=True
            elif(i.azimuth<max_sector+radar_rot_speed and i.azimuth>max_sector-radar_rot_speed):
                clockWise=False
            i.set(radar_speed,((i.azimuth+radar_rot_speed)%360) if clockWise else((i.azimuth-radar_rot_speed)%360) ,0)
        elif (radar_mode=="helical"):
            clockWise=False
            i.set(radar_speed,((i.azimuth+radar_rot_speed)%360) if clockWise else((i.azimuth-radar_rot_speed)%360) ,(i.elevation+0.07)%360)
        elif (radar_mode=="raster"):
            min_sector=30
            max_sector=250
            min_raster_sector=40
            max_raster_sector=-40
            if(i.azimuth<min_sector+radar_rot_speed and i.azimuth>min_sector-radar_rot_speed):
                clockWise=True
                if raster_clockWise:
                    raster_elevation-=raster_speed
                else:
                    raster_elevation+=raster_speed
            elif(i.azimuth<max_sector+radar_rot_speed and i.azimuth>max_sector-radar_rot_speed):
                clockWise=False
                if raster_clockWise:
                    raster_elevation-=raster_speed
                else:
                    raster_elevation+=raster_speed
            if(i.elevation<min_raster_sector+raster_speed and i.elevation>min_raster_sector-raster_speed):
                raster_clockWise=True
            elif(i.elevation<max_raster_sector+raster_speed and i.elevation>max_raster_sector-raster_speed):
                raster_clockWise=False
            i.set(radar_speed,((i.azimuth+radar_rot_speed)%360) if clockWise else((i.azimuth-radar_rot_speed)%360) ,raster_elevation)
        elif (radar_mode=="conical"):
            conical_theta+=conical_theta_speed
            i.set(radar_speed,180+(math.asin(conical_rad*math.cos(conical_theta*math.pi/180)/math.sqrt(conical_rad**2+conical_dis**2))*180/math.pi),math.asin(conical_rad*math.sin(conical_theta*math.pi/180)/math.sqrt(conical_rad**2+conical_dis**2))*180/math.pi)
        elif (radar_mode=="spiral"):
            conical_theta+=conical_theta_speed
            if(spiral_increse):
                conical_rad_spiral+=.005
            else:
                conical_rad_spiral-=.005
            if(abs(conical_rad_spiral)>conical_rad):
                spiral_increse=not spiral_increse
            i.set(radar_speed,180+(math.asin(abs(conical_rad_spiral)*math.cos(conical_theta*math.pi/180)/math.sqrt(conical_rad_spiral**2+conical_dis**2))*180/math.pi),math.asin(abs(conical_rad_spiral)*math.sin(conical_rad_spiral*math.pi/180)/math.sqrt(conical_rad**2+conical_dis**2))*180/math.pi)
        elif (radar_mode=="palmer"):
            min_sector=30
            max_sector=-40
            conical_theta+=conical_theta_speed
            print(conical_theta,palmer_azimuth)
            if(conical_theta>360):
                if(palmer_azimuth<min_sector+palmer_azimuth_speed//2 and palmer_azimuth>min_sector-palmer_azimuth_speed//2):
                    clockWise=True
                elif(palmer_azimuth<max_sector+palmer_azimuth_speed//2 and palmer_azimuth>max_sector-palmer_azimuth_speed//2):
                    clockWise=False
                conical_theta-=360
                if clockWise:
                    palmer_azimuth-=palmer_azimuth_speed
                else:
                    palmer_azimuth+=palmer_azimuth_speed
            i.set(radar_speed,180+palmer_azimuth+(math.asin(conical_rad*math.cos(conical_theta*math.pi/180)/math.sqrt(conical_rad**2+conical_dis**2))*180/math.pi),math.asin(conical_rad*math.sin(conical_theta*math.pi/180)/math.sqrt(conical_rad**2+conical_dis**2))*180/math.pi)
        elif (radar_mode=="palmerRaster"):
            print(palmer_azimuth,raster_elevation)
            min_sector=30
            max_sector=-40
            min_raster_sector=40
            max_raster_sector=-40
            conical_theta+=conical_theta_speed
            print(conical_theta,palmer_azimuth)
            if(conical_theta>360):
                if(palmer_azimuth<min_sector+palmer_azimuth_speed//2 and palmer_azimuth>min_sector-palmer_azimuth_speed//2):
                    clockWise=True
                    if raster_clockWise:
                        raster_elevation-=palmer_raster_speed
                    else:
                        raster_elevation+=palmer_raster_speed
                elif(palmer_azimuth<max_sector+palmer_azimuth_speed//2 and palmer_azimuth>max_sector-palmer_azimuth_speed//2):
                    clockWise=False
                    if raster_clockWise:
                        raster_elevation-=palmer_raster_speed
                    else:
                        raster_elevation+=palmer_raster_speed
                conical_theta-=360
                if clockWise:
                    palmer_azimuth-=palmer_azimuth_speed
                else:
                    palmer_azimuth+=palmer_azimuth_speed

                if(i.elevation<min_raster_sector+palmer_raster_speed and i.elevation>min_raster_sector-palmer_raster_speed):
                    raster_clockWise=True
                elif(i.elevation<max_raster_sector+palmer_raster_speed and i.elevation>max_raster_sector-palmer_raster_speed):
                    raster_clockWise=False
            i.set(radar_speed,180+palmer_azimuth+(math.asin(conical_rad*math.cos(conical_theta*math.pi/180)/math.sqrt(conical_rad**2+conical_dis**2))*180/math.pi),math.asin(conical_rad*math.sin(conical_theta*math.pi/180)/math.sqrt(conical_rad**2+conical_dis**2))*180/math.pi+raster_elevation)
        else:
            i.set(radar_speed,random.randint(heading_angle-heading_noise_range,heading_angle+heading_noise_range))
        if(tmp_i==0):
            Rx_t_table.append(radar_pos[0])
            Ry_t_table.append(radar_pos[1])
            Ra_t_table.append(i.azimuth)
        i.ontick()
        tmp_i+=1

    # Get,Set,Ontick of RWR
    for i in aircraft_0.onboard_systems:
        rwr_0.set()
        rwr_0.ontick(radar)
        t_temp=rwr_0.get()[0]

        # print(len(FallState))
        if(len(FallState)<3):
            if((t_temp[0] > -85) or (t_temp[1] > -85) or (t_temp[2] > -85) or (t_temp[3] > -85)):
                if(StateFlag1==0):
                    RiseState.append(idx-1-tolerance)
                    # print('Tick - ',idx)
                    # print(RiseState)
                    # print(FallState)
                    # cv2.waitKey(0)
                StateFlag1 = 1
                # print('High')
            else:
                if(StateFlag1==1):
                    FallState.append(idx-0+tolerance)
                    # print('Tick - ',idx)
                    # print(RiseState)
                    # print(FallState)
                    # cv2.waitKey(0)
                StateFlag1 = 0
                # print('Low')

            ResetFlag=1
            shiftPrevious = 0
            shiftCurrent = 0
        else:
            if(ResetFlag):
                if(abs((FallState[2]-FallState[1]) - (FallState[1]-FallState[0]))<2 and abs((RiseState[2]-RiseState[1]) - (RiseState[1]-RiseState[0]))<2):
                    ResetFlag = 0
                    pri = FallState[2]-FallState[1]
                    pulseWidth = FallState[2] - RiseState[2]#%pri
                    # print(pri,pulseWidth)
                    # cv2.waitKey(0)
                else:
                    ResetFlag = 0
                    FallState.pop(0)
                    RiseState.pop(0)
                    # print(FallState,RiseState)
                    # cv2.waitKey(0)
                shiftPrevious = 0
                shiftCurrent = 0

        if (((idx-(FallState[-1] if len(FallState)>0 else 0)-shiftCurrent)+tolerance+1)%pri > pulseWidth and ((idx-(FallState[-1] if len(FallState)>0 else 0)-shiftCurrent)+tolerance+1)%pri<pulseWidth+10):
            shiftCurrent = shiftPrevious
            A1_t_table.append(-90)
            A2_t_table.append(-90)
            A3_t_table.append(-90)
            A4_t_table.append(-90)

        elif (((idx-(FallState[-1] if len(FallState)>0 else 0)-shiftCurrent)+tolerance+1)%pri > pri - pulseWidth):
            if(StateFlag2 == 0):
                # print('Rise')
                # cv2.waitKey(0)
                pass
            StateFlag2 = 1
            A1_t_table.append(t_temp[0])
            A2_t_table.append(t_temp[1])
            A3_t_table.append(t_temp[2])
            A4_t_table.append(t_temp[3])

            if((t_temp[0] > -85) or (t_temp[1] > -85) or (t_temp[2] > -85) or (t_temp[3] > -85)):
                StateFlag3 += 1
        else:
            if(StateFlag2 == 1):
                # print('Fall',StateFlag3,idx)
                # cv2.waitKey(0)
                if(StateFlag3==0):
                    missingCount +=1
                    if(missingCount>2):
                        FallState.clear()
                        RiseState.clear()
                        pri = 123
                        pulseWidth = pri-0
                        shiftPrevious = 0
                        shiftCurrent = 0
                else:
                    if(StateFlag3<pulseWidth-3):
                        # print(StateFlag3,pulseWidth)
                        if((A1_t_table[-3] > -85) or (A2_t_table[-3] > -85) or (A3_t_table[-3] > -85) or (A4_t_table[-3] > -85)):
                            shiftPrevious += ( pulseWidth - StateFlag3 - 2 )
                            # pass
                        else:
                            shiftPrevious -= ( pulseWidth - StateFlag3 - 2 )
                            # pass
                        # cv2.waitKey(0)
                    missingCount = 0
            StateFlag2 = 0
            StateFlag3 = 0

            A1_t_table.append(-90)
            A2_t_table.append(-90)
            A3_t_table.append(-90)
            A4_t_table.append(-90)

            # A1_t_table.append(t_temp[0])
            # A2_t_table.append(t_temp[1])
            # A3_t_table.append(t_temp[2])
            # A4_t_table.append(t_temp[3])

    # Visualization
    img=np.zeros(img.shape,np.uint8)
    for j in range(receivers_n):
        cv2.line(img,airc_pos,(int(airc_pos[0]+(antenna_length*math.cos(math.radians(aircraft_0.heading+(360/receivers_n)*(1+2*j)/2)))),int(airc_pos[1]+(antenna_length*math.sin(math.radians(aircraft_0.heading+(360/receivers_n)*(1+2*j)/2))))),(0,0,255),2)
    cv2.circle(img,airc_pos,5,(0,255,0),-1)
    airc_head=(int(airc_pos[0]+(antenna_length*1.5*math.cos(math.radians(aircraft_0.heading))))
              ,int(airc_pos[1]+(antenna_length*math.sin(math.radians(aircraft_0.heading)))))
    cv2.line(img,airc_pos,airc_head,(0,255,0),4)
    #Aircraft waypoints
    # for j in range(len(aircraft_0.aircraftWaypoints)-1):
    #     cv2.line(img,aircraft_0.aircraftWaypoints[j],aircraft_0.aircraftWaypoints[j+1],(255,255,0),2)
    # cv2.line(img,aircraft_0.aircraftWaypoints[-1],aircraft_0.aircraftWaypoints[0],(255,255,0),2)
    for j in range(len(radar)):
        # adding the radars to the image
        radar_pos=(int(radar[j].position[0]),int(radar[j].position[1]))
        cv2.circle(img,radar_pos,7,(255,155,0),-1)
        cv2.putText(img,str(radar[j].id),(radar_pos[0]+20,radar_pos[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)
        cv2.line(img,radar_pos,(int(radar_pos[0]+(tmp_s_r*math.cos(math.radians(radar[j].azimuth)))),int(radar_pos[1]+(tmp_s_r*math.sin(math.radians(radar[j].azimuth))))),(0,0,255),2)
    image=cv2.putText(img,'tick number - '+str(idx),(img.shape[1]*6//10,img.shape[0]*1//10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('Visualisation',img)
    idx+=1
    # break
    if(play_flag):
        # if(idx<15 or (idx>92 and idx<98) or (idx>99 and idx<107)):
        if((idx>4 and idx<10)):
            quit_k=cv2.waitKey(play_back_speed) & 0xFF
            # quit_k=cv2.waitKey(0)&0xFF
        else:
            quit_k=cv2.waitKey(play_back_speed) & 0xFF
    else:
        quit_k = cv2.waitKey(0) & 0xFF
    clear_output(wait=True)
    if quit_k == ord('q'):
        break
    if quit_k == ord('p'):
        play_flag=False
    if quit_k == ord('l'):
        play_flag=True
# cv2.imwrite('Visualisation.jpg',img)
cv2.destroyAllWindows()

In [ ]:
# plt.figure(figsize=(24 , 2))
# plt.plot(A1_t_table,label='A1')
# plt.plot(A2_t_table,label='A2')
# plt.plot(A3_t_table,label='A3')
# plt.plot(A4_t_table,label='A4')
# for i in RiseState:
#     plt.scatter(i,-85, color = 'hotpink')
# for i in FallState:
#     plt.scatter(i,-85, color = '#88c999')
# plt.legend()
# plt.show()

# print('Main Table')
# t = PrettyTable(['Tick','Aircraft X', 'Aircraft Y','Radar X','Radar Y','Radar Azimuth','A1 ','A2'])
# for i in range(len(A1_t_table)):
#     t.add_row([i,Ax_t_table[i],Ay_t_table[i],Rx_t_table[i],Ry_t_table[i],Ra_t_table[i],A1_t_table[i],A2_t_table[i]])
# print(t)

In [ ]:
A1_t_peak, _ = find_peaks(A1_t_table)
A2_t_peak, _ = find_peaks(A2_t_table)
A3_t_peak, _ = find_peaks(A3_t_table)
A4_t_peak, _ = find_peaks(A4_t_table)
# t_peaks=A1_t_peak+A2_t_peak+A3_t_peak+A4_t_peak
t_peaks=list(A1_t_peak)+list(A2_t_peak)
t_peaks = sorted(set(t_peaks), key = lambda ele: list(t_peaks).count(ele))
t_peaks.sort()

# Visualization
print('Filtered Table just for detecting peaks in both A1 and A2')
t = PrettyTable(['Tick','Aircraft X', 'Aircraft Y','Radar X','Radar Y','Radar Azimuth','A1 ','A2'])
for i in t_peaks:
    t.add_row([i,Ax_t_table[i],Ay_t_table[i],Rx_t_table[i],Ry_t_table[i],Ra_t_table[i],A1_t_table[i],A2_t_table[i]])
print(t)

In [ ]:
# freqs=pdws[-1]['Frequency (GHz)'][0][:-1]
# freq_2=[]
# freq_3=[]
# for i in range(len(freqs)):
#     if(freqs[i]>=100):
#         print(freqs[i],end=',')
#     else:
#         print(freqs[i],end=' ,')
# print()
# print(end='  ')
# for i in range(len(freqs)-1):
#     temp=round(100/freqs[i],2)
#     if(temp>=100):
#         print(temp,end=',')
#     elif(temp>=10):
#         print(temp,end=' ,')
#     else:
#         print(temp,end=' , ')
#     freq_2.append(temp)
# print()
# print(end='  ')
# for i in range(len(freq_2)-1):
#     temp=freq_2[i]-freq_2[i+1]
#     if(temp>=100):
#         print(temp,end=',')
#     elif(temp>=10):
#         print(temp,end=' ,')
#     else:
#         print(temp,end=' , ')

In [ ]:
# # Display independent Pulse Train
# import matplotlib.pyplot as plt

# time_l=[*range(len(pulse_train_f[0]))]
# ci=0#8*len(time_l)//9
# cf=len(time_l)//1

# fig_x , fig_y = 25 , 2
# clrs= 'rgbcmyk'

# for i in pulse_train:
#     plt.figure(figsize=(fig_x , fig_y))
#     jdx=0
#     for j in i:
#         jdx+=1
#         plt.plot(time_l[ci:cf],j[ci:cf], color=clrs[jdx-1], label=str(jdx))
#     plt.legend()
#     plt.xlabel("independent input pulses")
#     # plt.ylim([0, ylim])
#     # plt.savefig("independent"+str(jdx)+".jpg")
#     plt.show()

In [ ]:
# # Display interleaved Pulse Train

# import matplotlib.pyplot as plt

# time_l=[*range(len(pulse_train_f[0]))]
# ci=0#len(time)//4
# cf=len(time_l)//1

# fig_x , fig_y = 25 , 2
# clrs= 'rgbcmyk'

# # for i in pulse_train_f:
# #     print(i)
# # for i in pulse_train_f:
# for i in pulse_train_f:
#     plt.figure(figsize=(fig_x , fig_y))
#     plt.plot(time_l[ci:cf],i[ci:cf], color=clrs[0])
#     plt.xlabel("interleaved pulses")
#     # plt.ylim([0, ylim])
#     # plt.savefig("independent"+str(jdx)+".jpg")
#     plt.show()
# # xt = pulse_train_f[1]
# # fxt = np.fft.fft(xt)
# # # fxt = np.real(fxt)
# # fft = (np.abs(fxt)**1)[:len(fxt)//2]
# # fft = fft[:]
# # plt.figure(figsize=(fig_x , fig_y))
# # plt.plot(time_l[ci:len(fft)],fft[ci:cf], color=clrs[0])
# # plt.title("Magnitude of the FT of x(t)")
# # plt.ylabel("Magnitude")
# # plt.xlabel("Frequency Index")
# # # plt.xticks(np.arange(0, len(fft), 50//fac))
# # plt.show()

In [ ]:
# # Aircraft and radar waypoints

# def draw_rectangle(event,x,y,flags,param):
#     global pt_clicked
#     if event == cv2.EVENT_LBUTTONDOWN:
#         pt1 = (x,y)
#         pt_clicked.append(pt1)
#         cv2.circle(frame, center=pt1, radius=3, color=(255,255,255), thickness=-1)

# cv2.namedWindow("Test", cv2.WINDOW_NORMAL)
# cv2.setMouseCallback('Test', draw_rectangle)
# resize=1.5
# cv2.resizeWindow("Test", int(img.shape[1]/resize), int(img.shape[0]/resize))

# pt_clicked=[]

# frame = np.zeros(img.shape, np.uint8)
# while True:
#     for i in radars:
#         radar_pos=(int(i.position[0]),int(i.position[1]))
#         heading_angle=int(math.degrees(math.atan2(airc_pos[1]-radar_pos[1],airc_pos[0]-radar_pos[0])))#+180
#         heading_noise_range=0
#         i.heading=random.randint(heading_angle-heading_noise_range,heading_angle+heading_noise_range)
#         cv2.circle(frame, radar_pos, 5, (255,0,0), -1)
#         # cv2.circle(frame, radar_pos, 1200, (255,0,255), 1)
#         cv2.line(frame, radar_pos,
#                  (int(radar_pos[0]+(tmp_s_r*math.cos(math.radians(i.heading)))),
#                   int(radar_pos[1]+(tmp_s_r*math.sin(math.radians(i.heading))))), (0,0,255), 2)
#     # for j in range(90):
#     #     cv2.circle(frame,(int(1400-(1300*math.cos(math.radians(j)))),int(1400-(1300*math.sin(math.radians(j))))), 5, (255,255,255), -1)
#     cv2.imshow('Test', frame)
#     qk=cv2.waitKey(1) & 0xFF
#     if qk == ord('q'):
#         print(pt_clicked)
#         break
#     if qk == ord('r'):
#         pt_clicked=[]
#         frame = np.zeros(img.shape, np.uint8)
#     if qk == ord('s'):
#         print(pt_clicked)
# cv2.destroyAllWindows()

In [ ]:
# tmp_list=[]
# for i in range(90):
#     tmp_list.append((int(1400-(1300*math.cos(math.radians(i)))),int(1400-(1300*math.sin(math.radians(i))))))
# print(tmp_list)

In [ ]:
# # Radar coordinates

# def draw_rectangle(event,x,y,flags,param):
#     global pt_clicked
#     if event == cv2.EVENT_LBUTTONDOWN:
#         pt1 = (float(x),float(y))
#         pt_clicked.append(pt1)
#         cv2.circle(frame, center=(int(pt1[0]),int(pt1[1])), radius=5, color=(0,0,255), thickness=-1)

# cv2.namedWindow('Test')
# cv2.setMouseCallback('Test', draw_rectangle)

# pt_clicked=[]

# frame = np.zeros(img.shape, np.uint8)
# while True:
#     cv2.imshow('Test', frame)
#     qk=cv2.waitKey(1) & 0xFF
#     if qk == ord('q'):
#         print(pt_clicked)
#         break
#     if qk == ord('r'):
#         pt_clicked=[]
#         frame = np.zeros(img.shape, np.uint8)
#     if qk == ord('s'):
#         print(pt_clicked)
# cv2.destroyAllWindows()

In [ ]:
# frame = np.zeros((500,2000,3), np.uint8)
# while True:
#     cv2.imshow('Test', frame)
#     qk=cv2.waitKey(1) & 0xFF
#     if qk == ord('q'):
#         break
# cv2.destroyAllWindows()

In [ ]:
tm = np.arange(0,360,1)

input_wave = np.cos(mult*np.radians(tm))
plt.polar(np.radians(tm)[:90//mult],input_wave[:90//mult])
plt.polar(np.radians(tm)[len(input_wave)-(90//mult):],input_wave[len(input_wave)-(90//mult):])
plt.show()